In [96]:
url1 = (r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json')

In [102]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

pd.set_option('display.max_rows', 400)

In [103]:

#This is live data from the our world in data github. This is the csv they put on the website. Covid19 data.
data=pd.read_json(url1)
data1 = pd.DataFrame(data).T
data1 = data1.astype(str)
data1 = data1[data1 != 'None']
data1['total_deaths'] = data1['total_deaths'].astype(float)
data1


,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
AFG,Asia,Afghanistan,2021-11-27,157190.0,19.0,46.571,7308.0,1.0,-7.857,3945.985,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
OWID_AFR,NaN,Africa,2021-11-27,8632315.0,8236.0,4794.714,222501.0,133.0,144.857,6284.965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,Europe,Albania,2021-11-27,199137.0,405.0,449.857,3085.0,8.0,7.143,69314.854,...,7.1,51.2,NaN,2.89,78.57,0.795,NaN,NaN,NaN,NaN
DZA,Africa,Algeria,2021-11-27,209980.0,163.0,163.0,6052.0,6.0,5.0,4706.317,...,0.7000000000000001,30.4,83.741,1.9,76.88,0.748,NaN,NaN,NaN,NaN
AND,Europe,Andorra,2021-11-27,16712.0,0.0,89.429,131.0,0.0,0.14300000000000002,216045.712,...,29.0,37.8,NaN,NaN,83.73,0.868,NaN,NaN,NaN,NaN
AGO,Africa,Angola,2021-11-27,65139.0,9.0,18.286,1733.0,0.0,0.429,1919.601,...,NaN,NaN,26.664,NaN,61.15,0.581,NaN,NaN,NaN,NaN
AIA,North America,Anguilla,2021-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,81.88,NaN,NaN,NaN,NaN,NaN
ATG,North America,Antigua and Barbuda,2021-11-27,4141.0,0.0,0.857,117.0,0.0,1.286,41943.522,...,NaN,NaN,NaN,3.8,77.02,0.778,NaN,NaN,NaN,NaN
ARG,South America,Argentina,2021-11-27,5325560.0,1521.0,1551.143,116517.0,12.0,20.429,116773.685,...,16.2,27.7,NaN,5.0,76.67,0.845,NaN,NaN,NaN,NaN
ARM,Asia,Armenia,2021-11-27,337522.0,517.0,562.714,7485.0,26.0,33.143,113715.446,...,1.5,52.1,94.043,4.2,75.09,0.776,NaN,NaN,NaN,NaN


In [76]:
#Attempted to create the dataframe from the url but I got a forbidden 403 error.
url2 = ('https://apps.who.int/gho/athena/api/GHO/HWF_000.json?filter=YEAR:2019&filter=text')
url2

'https://apps.who.int/gho/athena/api/GHO/HWF_000.json?filter=YEAR:2019&filter=text'

In [81]:
#Printed encoding of csv file to read the csv.
with open(r'C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\who_doctors.csv') as f:
        print(f.encoding)

cp1252


In [106]:
#Created the dataframe and cleaned the location column.
data2 = pd.read_csv(r'C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\who_doctors.csv', encoding ="cp1252")
data2 = data2[data2['Period'] == 2018]
data2['Location'] = data2['Location'].replace('Iran (Islamic Republic of)', 'Iran')
data2['Location'] = data2['Location'].replace('CÃ´te dâ€™Ivoire', "Cote d'Ivoire")
data2['Location'] = data2['Location'].replace('United States of America', 'United States')
data2['Location'] = data2['Location'].replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
data2['Location'] = data2['Location'].replace('Russian Federation', 'Russia')
data2['Location'] = data2['Location'].replace('Republic of Korea', 'South Korea')
data2

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
41,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MWI,Malawi,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,EN,2021-02-17T00:00:00.000Z
42,HWF_0001,"Medical doctors (per 10,000)",numeric,WPR,Western Pacific,Country,PNG,Papua New Guinea,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.70,NaN,NaN,EN,2021-02-17T00:00:00.000Z
43,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,SLE,Sierra Leone,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.74,NaN,NaN,EN,2021-02-17T00:00:00.000Z
44,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,ETH,Ethiopia,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.77,NaN,NaN,EN,2021-02-17T00:00:00.000Z
45,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,TGO,Togo,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.77,NaN,NaN,EN,2021-02-17T00:00:00.000Z
46,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BEN,Benin,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.79,NaN,NaN,EN,2021-02-17T00:00:00.000Z
47,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MOZ,Mozambique,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.84,NaN,NaN,EN,2021-02-17T00:00:00.000Z
48,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BFA,Burkina Faso,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.85,NaN,NaN,EN,2021-02-17T00:00:00.000Z
49,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,GHA,Ghana,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.96,NaN,NaN,EN,2021-02-17T00:00:00.000Z
50,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MLI,Mali,Year,2018,...,NaN,NaN,NaN,NaN,NaN,1.29,NaN,NaN,EN,2021-02-17T00:00:00.000Z


In [107]:
#Merge the who doctors dataset with the covid19 api.
merged_covid_docs = pd.merge(data1, data2, left_on=['location'], right_on=['Location'])
merged_covid_docs

,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,Europe,Albania,2021-11-27,199137.0,405.0,449.857,3085.0,8.0,7.143,69314.854,...,NaN,NaN,NaN,NaN,NaN,21.58,NaN,NaN,EN,2021-02-17T00:00:00.000Z
1,Africa,Algeria,2021-11-27,209980.0,163.0,163.0,6052.0,6.0,5.0,4706.317,...,NaN,NaN,NaN,NaN,NaN,17.19,NaN,NaN,EN,2021-02-17T00:00:00.000Z
2,Oceania,Australia,2021-11-27,207984.0,1235.0,1363.143,1994.0,4.0,7.143,8065.079,...,NaN,NaN,NaN,NaN,NaN,37.60,NaN,NaN,EN,2021-02-17T00:00:00.000Z
3,Europe,Austria,2021-11-27,1132805.0,11671.0,12890.571,12349.0,65.0,50.857,125267.719,...,NaN,NaN,NaN,NaN,NaN,52.11,NaN,NaN,EN,2021-02-17T00:00:00.000Z
4,Asia,Bangladesh,2021-11-27,1575579.0,155.0,241.429,27975.0,2.0,4.143,9474.118,...,NaN,NaN,NaN,NaN,NaN,5.81,NaN,NaN,EN,2021-02-17T00:00:00.000Z
5,Europe,Belgium,2021-11-27,1701633.0,0.0,17161.857,26840.0,0.0,38.857,146284.744,...,NaN,NaN,NaN,NaN,NaN,31.15,NaN,NaN,EN,2021-02-17T00:00:00.000Z
6,North America,Belize,2021-11-27,30165.0,0.0,94.857,570.0,0.0,1.714,74497.117,...,NaN,NaN,NaN,NaN,NaN,10.78,NaN,NaN,EN,2021-02-17T00:00:00.000Z
7,Africa,Benin,2021-11-27,24850.0,0.0,0.5710000000000001,161.0,0.0,0.0,1995.819,...,NaN,NaN,NaN,NaN,NaN,0.79,NaN,NaN,EN,2021-02-17T00:00:00.000Z
8,Asia,Bhutan,2021-11-27,2636.0,2.0,0.5710000000000001,3.0,0.0,0.0,3379.921,...,NaN,NaN,NaN,NaN,NaN,4.24,NaN,NaN,EN,2021-02-17T00:00:00.000Z
9,Africa,Botswana,2021-11-27,194909.0,0.0,66.286,2416.0,0.0,0.0,81305.585,...,NaN,NaN,NaN,NaN,NaN,2.89,NaN,NaN,EN,2021-02-17T00:00:00.000Z
